In [1]:
from ARC import ARC

train, test = ARC.load(387)

train[0]

`setup_ipynb.py` executed, (multi-output enabled for subsequent cell executions).


[
🔵 ⚫ ⚫
⚫ ⚫ ⚫
⚫ ⚫ 🔵
, 
🔵 ⚫ 🔴 🔵 ⚫ 🔴
🔴 ⚫ 🔴 🔴 ⚫ 🔴
🔴 ⚫ 🔵 🔴 ⚫ 🔵
🔵 ⚫ 🔴 🔵 ⚫ 🔴
🔴 ⚫ 🔴 🔴 ⚫ 🔴
🔴 ⚫ 🔵 🔴 ⚫ 🔵
]

In [2]:
train.overview()

X:  3x3   🔳  2px ⛓2  🔗2 ,  🔵  2px ⛓2  🔗2 ,  ⚫  7px ⛓1  🔗1 ,                  
Y:  6x6   🔳 24px ⛓3  🔗3 ,  🔵  8px ⛓8  🔗7 ,  ⚫ 12px ⛓2  🔗2 ,  🔴 16px ⛓6  🔗5 

X:  6x6   🔳  3px ⛓3  🔗3 ,  ⚫ 33px ⛓1  🔗1 ,  🟣  3px ⛓3  🔗3 ,                  
Y: 12x12  🔳 72px ⛓3  🔗3 ,  ⚫ 72px ⛓2  🔗2 ,  🟣 12px ⛓10 🔗10,  🔴 60px ⛓5  🔗5 

X:  2x2   🔳  1px ⛓1  🔗1 ,  ⚫  3px ⛓1  🔗1 ,  🟡  1px ⛓1  🔗1 ,                  
Y:  4x4   🔳  8px ⛓2  🔗2 ,  ⚫  8px ⛓2  🔗2 ,  🟡  4px ⛓4  🔗4 ,  🔴  4px ⛓4  🔗4 



In [3]:
# 👁: {k}x{k} -> {2k}.{2k}
# 👁: Designating 🌹 for X.non⚫, 🌹.Σ -> 4 x 🌹.Σ
# 👁: 🔴 is introduced, and is a multiple of 4
# 👁: pair[2] is small: .X is 2x2 with one 🟡 pixel, and .Y only adds 4 🔴 pixels.

# ⚡: Let's examine pair[2]

train[2]

[
⚫ 🟡
⚫ ⚫
, 
⚫ 🟡 ⚫ 🟡
⚫ 🔴 ⚫ 🔴
⚫ 🟡 ⚫ 🟡
⚫ 🔴 ⚫ 🔴
]

In [4]:
# 👁: There's a 2x2 tiling in Y

# ⚡: Is this ALWAYS true?

from Iter import Iter
from Supergrid import Supergrid

[
    Iter(Supergrid(Y, ny=2, nx=2)).unique.Σ
        for Y in train.Y
]


[1, 1, 1]

In [5]:
# 👑: If we can generate a single tile from X, we're done!

# 👶: Let's simplify by recolouring and setting Y to a single tile.

from Pairs import Pairs

def simplify(pair):
    X, Y = pair
    C = str(X.glyphs - '⚫')
    s = f'{C}->🌹'
    return X[s], Y.split(2, 2)[0][0][s]

train_ = Pairs(simplify @ train)

train_

[[
🌹 ⚫ ⚫
⚫ ⚫ ⚫
⚫ ⚫ 🌹
, 
🌹 ⚫ 🔴
🔴 ⚫ 🔴
🔴 ⚫ 🌹
], [
⚫ 🌹 ⚫ ⚫ ⚫ ⚫
⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ ⚫ ⚫ ⚫ ⚫
⚫ ⚫ ⚫ ⚫ ⚫ ⚫
🌹 ⚫ ⚫ ⚫ ⚫ 🌹
⚫ ⚫ ⚫ ⚫ ⚫ ⚫
, 
🔴 🌹 ⚫ ⚫ ⚫ 🔴
🔴 🔴 ⚫ ⚫ ⚫ 🔴
🔴 🔴 ⚫ ⚫ ⚫ 🔴
🔴 🔴 ⚫ ⚫ ⚫ 🔴
🌹 🔴 ⚫ ⚫ ⚫ 🌹
🔴 🔴 ⚫ ⚫ ⚫ 🔴
], [
⚫ 🌹
⚫ ⚫
, 
⚫ 🌹
⚫ 🔴
]]

In [6]:
# 👁: In each col containing 🌹, ⚫ -> 🔴

def solve(X):
    tile = X.copy
    for col in tile.cols:
        if col.any:
            col.send('⚫->🔴', in_place=True)

    # Allocate a fresh Grid object of desired shape
    Y = X.upscale(2)

    Supergrid(Y, ny=2, nx=2)[:] = tile

    return Y

[Y @ solve(X)  for (X, Y) in train | test]

[True, True, True, True]